# HuggingFace Model API

In [8]:

from langchain.llms.base import LLM
from typing import Optional, List, Dict
import os
import requests

class HuggingFaceAPILLM(LLM):
    api_url: str = "https://router.huggingface.co/v1/chat/completions"
    api_key: str = os.environ["HF_TOKEN"]
    model: str = "meta-llama/Meta-Llama-3-8B-Instruct:novita"

    @property
    def _llm_type(self) -> str:
        return "huggingface_api"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {"Authorization": f"Bearer {self.api_key}"}
        payload = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}]
        }
        response = requests.post(self.api_url, headers=headers, json=payload)
        result = response.json()
        return result["choices"][0]["message"]["content"]


In [9]:
# Instantiate
llm = HuggingFaceAPILLM()


# Chunking the Text

In [19]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
import glob

data_dir = "../data/processed/"
txt_files = glob.glob(os.path.join(data_dir, "*.txt"))

documents = []
for file in txt_files:
    loader = TextLoader(file)
    docs = loader.load()
    documents.extend(docs)

# %%
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)


len(chunks)


2691

# Generating and storing Embeddings

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [13]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(chunks, embeddings_model)
vector_store.save_local("../faiss_index")  # stores locally


# Prompt Template


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """
You are an expert on laws, codes, and acts. Answer like a lawyer helping a common citizen regarding laws.

Context:
{context}

Question:
{question}

Answer concisely and clearly.
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)


# Retrieval Chain

In [15]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

# %%
query = "How do i file for bankcrupcy of my company"
answer = qa.run(query)
answer

/tmp/ipykernel_15723/2827365189.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run(query)


"To file for bankruptcy of your company, follow these steps:\n\n1. **Determine if you qualify**: Check if you or your creditors meet the eligibility criteria for filing a bankruptcy petition (typically, a creditor or group of creditors with at least 25% of the total debts or more than ₹500,000).\n2. **Gather necessary documents**: Collect financial statements, tax returns, and other relevant documents that show your company's financial situation.\n3. **Choose the right type of bankruptcy**: Decide whether you want to file for **Insolvency** (if your company is unable to pay its debts) or **Liquidation** (if your company is insolvent and needs to be wound up).\n4. **Prepare the petition**: File a petition in the court, accompanied by the required documents, including:\n\t* A statement of your company's financial position\n\t* A list of creditors and their claims\n\t* A plan for handling the bankruptcy proceedings\n5. **Submit the petition**: File the petition with the court and pay the 